# LLM 에이전트 협업 패턴 구현 실습

안녕하세요! Google ADK 가이드입니다.

이 노트북에서는 세 가지 주요 에이전트 협업 패턴(순차적, 계층적, 대화형)을 코드로 구현하는 방법을 다룹니다. 우리는 두 가지 주요 프레임워크인 **LangGraph**와 **Google ADK(Agent Development Kit)**를 사용하여 각 패턴을 어떻게 구현할 수 있는지 살펴볼 것입니다.

## 학습 목표
1.  **순차적(Sequential) 패턴**: 하나의 에이전트 출력이 다음 에이전트의 입력으로 직접 이어지는 파이프라인 구조를 구현합니다.
2.  **계층적(Hierarchical) 패턴**: 중앙의 관리자(Manager) 에이전트가 작업을 분해하고 여러 작업자(Worker) 에이전트에게 분배하는 구조를 구현합니다.
3.  **대화형(Conversational) 패턴**: 여러 에이전트가 서로의 작업을 비평하고 개선하며 아이디어를 발전시키는 동적인 상호작용 구조를 구현합니다.

In [ ]:
import os
import getpass

# API 키를 환경 변수로 설정합니다.
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = (
        "your-api-key"
    )

# 1. LangGraph 기반 구현

LangGraph의 각 노드(node)는 에이전트나 도구를 나타내고, 엣지(edge)는 노드 간의 제어 흐름을 정의합니다. 이를 통해 다양한 에이전트 협업 패턴을 유연하게 구현할 수 있습니다.

### 1-1. LangGraph 기반 순차적 패턴 구현

**시나리오**: '여행 계획'이라는 주제에 대한 블로그 글을 작성하는 순차적 프로세스를 만듭니다.
1.  `topic_researcher`: 주어진 주제에 대한 리서치를 수행하여 개요를 작성합니다.
2.  `content_writer`: 개요를 바탕으로 블로그 글의 초안을 작성합니다.
3.  `content_reviewer`: 작성된 초안을 검토하고 최종본을 만듭니다.

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
# 상태 정의: 그래프의 각 노드가 공유하고 수정할 데이터 구조
class BlogWorkflowState(TypedDict):
    topic: str
    outline: str
    draft: str
    final_content: str


# 에이전트(노드) 함수 정의
def topic_researcher(state):
    print("--- 순차적 패턴: 1. 주제 리서처 실행 ---")
    topic = state.get("topic")
    prompt = f"'{topic}'에 대한 블로그 글의 개요를 작성해줘."
    outline = llm.invoke(prompt).content
    print(f"생성된 개요: \n{outline[:200]}...")
    return {"outline": outline}


def content_writer(state):
    print("\n--- 순차적 패턴: 2. 콘텐츠 작성자 실행 ---")
    outline = state.get("outline")
    prompt = f"다음 개요를 바탕으로 블로그 글의 초안을 작성해줘:\n\n{outline}"
    draft = llm.invoke(prompt).content
    print(f"생성된 초안: \n{draft[:200]}...")
    return {"draft": draft}


def content_reviewer(state):
    print("\n--- 순차적 패턴: 3. 콘텐츠 검토자 실행 ---")
    draft = state.get("draft")
    prompt = f"다음 초안을 검토하고 문법과 스타일을 수정해서 최종 블로그 글로 만들어줘:\n\n{draft}"
    final_content = llm.invoke(prompt).content
    print(f"생성된 최종본: \n{final_content[:200]}...")
    return {"final_content": final_content}


# 그래프 생성 및 노드/엣지 정의
workflow = StateGraph(BlogWorkflowState)

workflow.add_node("researcher", topic_researcher)
workflow.add_node("writer", content_writer)
workflow.add_node("reviewer", content_reviewer)

# 엣지 연결 (순차적)
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", "reviewer")
workflow.add_edge("reviewer", END)

sequential_app = workflow.compile()

# 순차적 패턴 실행
inputs = {"topic": "LangGraph를 활용한 AI 에이전트 구축 방법"}
result = sequential_app.invoke(inputs)

print("\n--- 최종 결과 ---")
print(result["final_content"])

### 1-2. LangGraph 기반 계층적 패턴 구현

**시나리오**: 사용자가 '서울의 날씨와 그에 맞는 옷차림'을 질문하면, `manager_agent`가 이를 분석하여 두 개의 하위 작업으로 나누고, 각각 `weather_agent`와 `fashion_agent`에게 작업을 할당합니다. 마지막으로 두 작업자의 결과를 종합하여 최종 답변을 생성합니다.

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
import json


@tool
def get_weather(city: str):
    """특정 도시의 현재 날씨 정보를 가져옵니다."""
    if city == "서울":
        return "맑음, 25도"
    return "알 수 없음"


@tool
def get_fashion_advice(weather: str):
    """날씨 정보에 기반한 옷차림을 추천합니다."""
    if "맑음" in weather and "25도" in weather:
        return "반팔과 얇은 가디건을 추천합니다."
    return "날씨에 맞는 옷을 입으세요."


tools = [get_weather, get_fashion_advice]
llm_with_tools = llm.bind_tools(tools)


# 계층적 패턴의 상태
class HierarchicalState(TypedDict):
    messages: Annotated[list, operator.add]


def manager_agent(state):
    print("--- 계층적 패턴: 1. 매니저 에이전트 실행 ---")
    # LLM을 호출하여 어떤 도구를 사용할지 결정
    response = llm_with_tools.invoke(state["messages"])
    print(f"매니저 결정: {response.tool_calls}")
    return {"messages": [response]}


def worker_agents(state):
    print("--- 계층적 패턴: 2. 워커 에이전트(도구) 실행 ---")
    last_message = state["messages"][-1]
    tool_outputs = []
    for tool_call in last_message.tool_calls:
        if tool_call["name"] == "get_weather":
            result = get_weather.invoke(tool_call["args"])
        elif tool_call["name"] == "get_fashion_advice":
            result = get_fashion_advice.invoke(tool_call["args"])
        else:
            result = "도구를 찾을 수 없습니다."

        print(f"도구 '{tool_call['name']}' 실행 결과: {result}")
        tool_outputs.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))
    return {"messages": tool_outputs}


def router(state):
    # 마지막 메시지에 tool_calls가 있으면 worker를, 없으면 END를 반환
    if state["messages"][-1].tool_calls:
        return "worker"
    else:
        return "__end__"


# 계층적 그래프 정의
hierarchical_workflow = StateGraph(HierarchicalState)
hierarchical_workflow.add_node("manager", manager_agent)
hierarchical_workflow.add_node("worker", worker_agents)

hierarchical_workflow.set_entry_point("manager")
hierarchical_workflow.add_conditional_edges("manager", router)
hierarchical_workflow.add_edge("worker", "manager")

hierarchical_app = hierarchical_workflow.compile()

In [ ]:
# 계층적 패턴 실행
initial_state = {"messages": [HumanMessage(content="오늘 서울 날씨랑 옷차림 추천해줘")]}
final_state = hierarchical_app.invoke(initial_state)

print("\n--- 최종 결과 ---")
print(final_state["messages"][-1].content)

### 1-3. LangGraph 기반 대화형 패턴 구현

**가능성 분석**: LangGraph의 가장 큰 장점 중 하나는 **순환(cycle)**을 포함한 그래프를 만들 수 있다는 것입니다. 이는 대화형 패턴을 구현하는 데 매우 적합합니다. 특정 조건에 따라 노드 간의 실행을 반복하며 결과를 점진적으로 개선할 수 있습니다.

**시나리오**: `writer_agent`가 글의 초안을 작성하면, `critic_agent`가 이를 평가하고 피드백을 줍니다. 이 피드백이 긍정적일 때까지(또는 최대 2회 반복) 글을 수정하는 과정을 반복합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class CriticResponse(BaseModel):
    is_good_enough: bool = Field(description="초안이 충분히 좋은지 여부")
    feedback: str = Field(description="개선을 위한 구체적인 피드백")


# 대화형 패턴의 상태
class ConversationalState(TypedDict):
    topic: str
    draft: str
    feedback: str
    is_good_enough: bool
    iteration: int


def writer_agent(state):
    print(f"\n--- 대화형 패턴: {state['iteration'] + 1}번째 작성 시작 ---")
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "당신은 창의적인 작가입니다."),
            ("human", "'{topic}'에 대한 짧은 글을 작성해 주세요."),
            ("ai", "{draft}"),
            ("human", "다음 피드백을 반영하여 글을 수정해 주세요: {feedback}"),
        ]
    ).partial(topic=state["topic"], draft=state.get("draft", ""))

    response = llm.invoke(prompt.format(feedback=state.get("feedback", "")))
    print(f"수정된 초안: \n{response.content[:200]}...")
    return {"draft": response.content, "iteration": state["iteration"] + 1}


def critic_agent(state):
    print("--- 대화형 패턴: 비평가 실행 ---")
    critic_llm = llm.with_structured_output(CriticResponse)
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "당신은 날카로운 문학 비평가입니다. 초안이 충분히 좋은지, 아니면 개선이 필요한지 판단하세요."),
            ("human", "다음 초안을 평가해 주세요: \n\n{draft}"),
        ]
    )
    response = critic_llm.invoke(prompt.format(draft=state["draft"]))
    print(f"비평가 피드백: {response.feedback} (승인: {response.is_good_enough})")
    return {"feedback": response.feedback, "is_good_enough": response.is_good_enough}


def should_continue(state):
    if state["iteration"] >= 2 or state.get("is_good_enough"):
        return END
    return "writer"


# 대화형 그래프 정의
conversational_workflow = StateGraph(ConversationalState)
conversational_workflow.add_node("writer", writer_agent)
conversational_workflow.add_node("critic", critic_agent)

conversational_workflow.set_entry_point("writer")
conversational_workflow.add_edge("writer", "critic")
conversational_workflow.add_conditional_edges("critic", should_continue)

conversational_app = conversational_workflow.compile()

conversational_app

In [ ]:
# 대화형 패턴 실행
initial_conv_state = {"topic": "인공지능과 함께하는 미래", "iteration": 0}
final_conv_result = conversational_app.invoke(initial_conv_state)

print("\n--- 최종 결과 ---")
print(final_conv_result["draft"])

---

## 1-4. [TODO] 지시에 따라 LangGraph로 계층적 패턴 구현하기

**과제**: '3일간의 제주도 여행 계획'을 세우는 계층적 에이전트 시스템을 만들어 보세요.

**지시사항**:
1.  **상태(State)**: 여행 계획에 필요한 정보(`destination`, `duration`, `messages`, `final_plan`)를 포함하는 `TravelPlanState`를 정의하세요.
2.  **도구(Tools)**: `recommend_activities`와 `recommend_restaurants` 두 개의 도구를 `@tool` 데코레이터를 사용하여 만드세요. API를 호출할 필요 없이, 간단한 문자열을 반환하도록 구현합니다.
3.  **에이전트(Nodes)**:
    -   `manager_agent`: 사용자의 요청을 받아 어떤 도구(작업자)를 호출할지 결정합니다.
    -   `worker_node`: `manager_agent`의 결정을 받아 도구를 실행하고, 그 결과를 `ToolMessage` 형태로 상태에 추가합니다.
4.  **라우터(Router)**: `manager_agent`의 출력에 `tool_calls`가 있는지, 그리고 모든 필요한 도구가 호출되었는지 확인하여 `worker_node`로 라우팅할지, 최종 계획을 생성하는 `create_final_plan` 노드로 라우팅할지 결정하는 함수를 작성하세요.
5.  **최종 계획 생성 노드**: 모든 정보가 수집되면, `create_final_plan` 노드에서 최종 여행 계획을 종합하여 출력합니다.
6.  **그래프 조립**: 위 요소들을 사용하여 `StateGraph`를 만들고, 노드와 조건부 엣지를 추가하여 전체 워크플로우를 완성하세요.

In [ ]:
# [TODO] 필요한 라이브러리 import
from typing import List
from langchain_core.messages import AIMessage


# [TODO] 1. 상태(State) 정의
class TravelPlanState(TypedDict):
    destination: str
    duration: int
    messages: Annotated[list, operator.add]
    final_plan: str


# [TODO] 2. 도구(Tools) 정의
@tool
def recommend_activities(destination: str, duration: int):
    """주어진 목적지와 기간에 맞는 활동을 추천합니다."""
    return f"{destination}에서의 {duration}일 동안의 활동 추천: 성산일출봉, 쇠소깍, 협재 해수욕장"


@tool
def recommend_restaurants(destination: str):
    """주어진 목적지에 맞는 맛집을 추천합니다."""
    return f"{destination} 맛집 추천: 흑돼지 전문점, 해물라면 맛집, 오션뷰 카페"


# LLM 및 도구 바인딩
travel_tools = [recommend_activities, recommend_restaurants]
travel_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(travel_tools)


# [TODO] 3. 에이전트(Nodes) 정의
def manager_agent_todo(state):
    print("--- TODO: 매니저 에이전트 실행 ---")
    ## TODO ##
    # 상태에서 메시지를 가져와 LLM을 호출하고, 그 결과를 'messages' 키로 반환하세요.
    response = travel_llm.invoke(state["messages"])
    return {"messages": [response]}


def worker_node_todo(state):
    print("--- TODO: 워커 노드 실행 ---")
    ## TODO ##
    # 마지막 메시지에서 tool_calls를 추출하고, 각 tool_call에 맞는 도구를 실행하세요.
    # 실행 결과를 ToolMessage 리스트로 만들어 'messages' 키로 반환하세요.
    last_message = state["messages"][-1]
    tool_outputs = []
    for tool_call in last_message.tool_calls:
        if tool_call["name"] == "recommend_activities":
            result = recommend_activities.invoke(tool_call["args"])
        elif tool_call["name"] == "recommend_restaurants":
            result = recommend_restaurants.invoke(tool_call["args"])
        else:
            result = "도구를 찾을 수 없습니다."
        tool_outputs.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))
    return {"messages": tool_outputs}


def create_final_plan_node(state):
    print("--- TODO: 최종 계획 생성 ---")
    final_prompt = (
        "모든 정보를 종합하여 다음 형식에 맞춰 3일간의 제주도 여행 계획을 세워주세요.\n"
        "- 1일차: [활동], [식당]\n"
        "- 2일차: [활동], [식당]\n"
        "- 3일차: [활동], [식당]"
    )
    final_plan = llm.invoke(state["messages"] + [HumanMessage(content=final_prompt)]).content
    return {"final_plan": final_plan}


# [TODO] 4. 라우터(Router) 정의
def router_todo(state):
    ## TODO ##
    # 마지막 메시지에 tool_calls가 있으면 'worker'를 반환합니다.
    # 모든 도구가 호출되었다고 판단되면(예: ToolMessage가 2개 이상 쌓이면) 'create_final_plan'을 반환합니다.
    # 그렇지 않으면 다시 'manager'를 호출합니다.
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "worker"

    tool_message_count = sum(1 for msg in state["messages"] if isinstance(msg, ToolMessage))
    if tool_message_count < len(travel_tools):
        return "manager"
    else:
        return "create_final_plan"


# [TODO] 6. 그래프 조립
todo_workflow = StateGraph(TravelPlanState)

## TODO ##
# add_node를 사용하여 위에서 정의한 노드들을 추가하세요.
todo_workflow.add_node("manager", manager_agent_todo)
todo_workflow.add_node("worker", worker_node_todo)
todo_workflow.add_node("create_final_plan", create_final_plan_node)

## TODO ##
# set_entry_point를 설정하고, add_conditional_edges와 add_edge를 사용하여 워크플로우를 완성하세요.
todo_workflow.set_entry_point("manager")
todo_workflow.add_conditional_edges(
    "manager", router_todo, {"worker": "worker", "create_final_plan": "create_final_plan", "manager": "manager"}
)
todo_workflow.add_edge("worker", "manager")
todo_workflow.add_edge("create_final_plan", END)

todo_app = todo_workflow.compile()

todo_app

In [ ]:
# 실행
initial_request = {
    "destination": "제주도",
    "duration": 3,
    "messages": [HumanMessage(content="제주도로 3일간 여행가려고 하는데, 활동이랑 맛집 추천해줘.")],
}

todo_result = todo_app.invoke(initial_request)

print("\n--- [TODO] 최종 결과 ---")
print(todo_result["final_plan"])

---

# 2. Google ADK 기반 구현

Google ADK(Agent Development Kit)는 에이전트와 도구를 선언적으로 정의하고, 이들의 관계를 설정하여 멀티 에이전트 시스템을 쉽게 구축할 수 있도록 지원하는 프레임워크입니다. Python 코드로 에이전트를 직접 정의하거나, YAML 설정을 통해 구조를 정의할 수 있습니다.

### 2-1. Google ADK 기반 순차적 & 계층적 패턴 구현

ADK에서는 `SequentialAgent` 클래스를 사용하여 여러 에이전트를 순차적으로 실행할 수 있습니다.

In [ ]:
import asyncio
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


async def run_adk_sequential(prompt: str):
    # LiteLlm 모델 생성
    model = LiteLlm(model="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

    # 순차적 에이전트들 생성
    researcher = LlmAgent(
        name="researcher", model=model, instruction="주어진 주제에 대한 블로그 글 개요를 작성해주세요."
    )

    writer = LlmAgent(name="writer", model=model, instruction="주어진 개요를 바탕으로 블로그 글 초안을 작성해주세요.")

    reviewer = LlmAgent(name="reviewer", model=model, instruction="주어진 초안을 검토하고 최종본으로 만들어주세요.")

    # SequentialAgent 생성
    sequential_agent = SequentialAgent(name="BlogWritingPipeline", sub_agents=[researcher, writer, reviewer])

    # 세션 서비스 및 러너 설정
    session_service = InMemorySessionService()
    runner = Runner(agent=sequential_agent, app_name="blog_app", session_service=session_service)

    # 세션 생성 및 실행
    session = await session_service.create_session(app_name="blog_app", user_id="user1")

    print("--- ADK 순차적 패턴 실행 ---")

    results = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part.from_text(text=prompt)]),
    ):
        if hasattr(event, "content") and event.content:
            results.append(str(event.content))

    for i, result in enumerate(results):
        print(f"\n=== {runner.agent.sub_agents[i].name} 결과 ===")
        print(result[:200] + "...")

    print(f"\n=== 최종 결과 ===")
    print(results[-1][:200] + "...")

    return True

In [ ]:
# ADK 순차적 패턴 실행
try:
    result = await run_adk_sequential(prompt="양자 컴퓨팅의 원리와 응용")
    print("✓ ADK 순차적 패턴 실행 완료")
except Exception as e:
    print(f"✗ 실행 오류: {e}")

### 2-1. Google ADK 기반 순차적 & 계층적 패턴 구현

ADK에서는 `ParallelAgent` 클래스를 사용하여 여러 에이전트를 병렬 실행할 수 있습니다.

In [ ]:
async def run_adk_parallel(prompt: str):
    # 병렬 패턴
    model = LiteLlm(model="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

    # 병렬 에이전트들
    from google.adk.agents import ParallelAgent

    weather_agent = LlmAgent(
        name="weather_agent",
        model=model,
        instruction="서울의 날씨 정보를 제공해주세요. 실제 데이터는 없으니 적절히 생성하세요.",
    )

    fashion_agent = LlmAgent(name="fashion_agent", model=model, instruction="주어진 날씨에 맞는 옷차림을 추천해주세요.")

    # ParallelAgent 생성
    parallel_agent = ParallelAgent(name="WeatherFashionTeam", sub_agents=[weather_agent, fashion_agent])

    session_service = InMemorySessionService()
    runner = Runner(agent=parallel_agent, app_name="weather_app", session_service=session_service)

    session = await session_service.create_session(app_name="weather_app", user_id="user1")

    print("--- ADK 계층적(병렬) 패턴 실행 ---")

    results = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part.from_text(text=prompt)]),
    ):
        if hasattr(event, "content") and event.content:
            results.append(str(event.content))

    for i, result in enumerate(results):
        print(f"\n=== {runner.agent.sub_agents[i].name} 결과 ===")
        print(result[:200] + "...")

    return True

In [ ]:
# ADK 계층적(병렬) 패턴 실행
try:
    result = await run_adk_parallel("오늘 서울 날씨와 봄철 옷차림 추천")
    print("✓ ADK 계층적(병렬) 패턴 실행 완료")
except Exception as e:
    print(f"✗ 실행 오류: {e}")

In [ ]:
# ADK 계층적(병렬) 패턴 실행
try:
    result = await run_adk_parallel("옷차림 추천")
    print("✓ ADK 계층적(병렬) 패턴 실행 완료")
except Exception as e:
    print(f"✗ 실행 오류: {e}")

### 2-3. Google ADK 기반 루프 패턴 구현

**가능성 분석**: ADK는 `LoopAgent`를 제공하여 대화형 패턴과 같은 반복적인 워크플로우를 구현할 수 있습니다. `LoopAgent`는 내부에 정의된 에이전트(들)를 지정된 횟수만큼 또는 특정 조건이 충족될 때까지 반복 실행합니다.

**시나리오**: `LoopAgent`를 사용하여 작가와 비평가 에이전트의 상호작용을 2회 반복합니다.

In [ ]:
async def run_adk_loop():
    # 대화형(루프) 패턴
    model = LiteLlm(model="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

    from google.adk.agents import LoopAgent

    # 작가 에이전트
    writer_agent = LlmAgent(name="writer", model=model, instruction="주어진 주제로 짧은 글을 작성하고 개선하세요.")

    # LoopAgent 생성 (최대 2회 반복)
    loop_agent = LoopAgent(name="WriterLoop", sub_agents=[writer_agent], max_iterations=2)

    session_service = InMemorySessionService()
    runner = Runner(agent=loop_agent, app_name="writer_app", session_service=session_service)

    session = await session_service.create_session(app_name="writer_app", user_id="user1")

    print("--- ADK 대화형(루프) 패턴 실행 ---")

    results = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part.from_text(text="인공지능과 함께하는 미래")]),
    ):
        if hasattr(event, "content") and event.content:
            results.append(str(event.content))

    print(f"총 {len(results)}개의 응답 생성")
    if results:
        final_result = results[-1]
        print(f"최종 결과 (처음 200자): {final_result[:200]}...")

    return True


# ADK 대화형(루프) 패턴 실행
try:
    result = await run_adk_loop()
    print("✓ ADK 대화형(루프) 패턴 실행 완료")
except Exception as e:
    print(f"✗ 실행 오류: {e}")

---

## 2-4. [TODO] 지시에 따라 ADK로 순차적 패턴 구현하기

**과제**: `SequentialAgent`를 사용하여, 먼저 창의적인 이야기 제목을 생성하고, 그 제목을 바탕으로 짧은 이야기를 작성하는 순차적 에이전트를 만들어 보세요.

**지시사항**:
1.  `title_agent`: '우주를 여행하는 고양이'라는 주제로 흥미로운 제목을 생성하는 `LlmAgent`를 만드세요.
2.  `story_writer_agent`: 이전 단계에서 생성된 제목(`{input}`)을 받아 짧은 이야기를 작성하는 `LlmAgent`를 만드세요.
3.  `sequential_story_agent`: 위 두 에이전트를 올바른 순서로 실행하는 `SequentialAgent`를 정의하세요.
4.  `run_agent`를 사용하여 최종 결과물을 출력하세요.

In [ ]:
async def solve_todo_2_4():
    """[TODO] ADK로 순차적 패턴 구현: 제목 → 이야기 작성"""
    model = LiteLlm(model="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

    # 1. 제목 생성 에이전트
    title_agent = LlmAgent(
        # YOUR CODE
    )

    # 2. 이야기 작성 에이전트
    story_writer_agent = LlmAgent(
        # YOUR CODE
    )

    # 3. 순차적 에이전트
    sequential_story_agent = SequentialAgent(
        # YOUR CODE
    )

    # 실행 환경 설정
    session_service = InMemorySessionService()
    runner = Runner(agent=sequential_story_agent, app_name="story_app", session_service=session_service)

    session = await session_service.create_session(app_name="story_app", user_id="user1")

    print("--- [TODO] ADK 순차적 패턴 실행 ---")

    results = []
    async for event in runner.run_async(
        user_id=session.user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part.from_text(text="우주를 여행하는 고양이")]),
    ):
        if hasattr(event, "content") and event.content:
            results.append(str(event.content))

    print(f"\n--- [TODO] 최종 결과 ---")
    if results:
        final_result = results[-1]
        print(f"완성된 이야기:\n{final_result[:500]}...")

    return True


# [TODO] 실행
try:
    result = await solve_todo_2_4()
    print("✓ [TODO] 2-4 ADK 순차적 패턴 완료")
except Exception as e:
    print(f"✗ 실행 오류: {e}")